In [ ]:
! pip install -q langchain_community tiktoken langchain-openai langchainhub langchain

# **1 - Query Rewriting: Reformulating queries to improve retrieval.**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os
from dotenv import load_dotenv
from google.colab import userdata
import openai

# Load environment variables from a .env file
load_dotenv()
# Set the OpenAI API key environment variable
openai.key = userdata.get('openai_api')
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api')
model = ChatOpenAI(temperature=0,model_name= 'gpt-4o-mini', max_tokens=4000)


In [ ]:

# Create a prompt template for query rewriting
query_rewrite_template = """Bạn là trợ lý AI được giao nhiệm vụ xây dựng lại các truy vấn của người dùng để cải thiện khả năng truy xuất trong hệ thống RAG.
Với truy vấn ban đầu, hãy viết lại để cụ thể hơn, chi tiết hơn và có khả năng truy xuất thông tin có liên quan hơn.

Truy vấn ban đầu: {query}

Truy vấn được viết lại:"""

query_rewrite_prompt = PromptTemplate(
    input_variables=["query"],
    template=query_rewrite_template
)

# Create an LLMChain for query rewriting
query_rewriter = query_rewrite_prompt | model

def rewrite_query(original_query):
    """
    Rewrite the original query to improve retrieval.

    Args:
    original_query (str): The original user query

    Returns:
    str: The rewritten query
    """
    response = query_rewriter.invoke(original_query)
    return response.content

In [ ]:
# example query over the understanding climate change dataset
original_query = "Học tiếng Anh sao cho hiệu quả?"
rewritten_query = rewrite_query(original_query)
print("Original query:", original_query)
print("\nRewritten query:", rewritten_query)

Original query: Học tiếng Anh sao cho hiệu quả?

Rewritten query: Truy vấn được viết lại: "Những phương pháp và kỹ thuật hiệu quả nhất để học tiếng Anh là gì? Tôi cần các gợi ý cụ thể về cách cải thiện kỹ năng nghe, nói, đọc và viết trong tiếng Anh, cũng như các tài nguyên học tập hữu ích."


# **2 - Step-back Prompting: Making queries more generally for better context retrieval.**

In [ ]:
step_back_prompt = """Bạn là trợ lý AI được giao nhiệm vụ tạo các truy vấn rộng hơn, tổng quát hơn để cải thiện việc truy xuất ngữ cảnh trong hệ thống RAG.
Với truy vấn ban đầu, hãy tạo một truy vấn bước lùi tổng quát hơn và có thể giúp truy xuất thông tin nền có liên quan.

Truy vấn ban đầu: {original_query}

Truy vấn bước lùi:"""

step_back_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=step_back_prompt
)

step_back_chain = LLMChain(llm=model, prompt=step_back_prompt)
def generate_step_back_query(original_query):
    """
    Generate a step-back query to retrieve broader context.

    Args:
    original_query (str): The original user query

    Returns:
    str: The step-back query
    """
    response = step_back_chain.invoke(original_query)
    return response


<ipython-input-9-2390954b59fb>:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  step_back_chain = LLMChain(llm=model, prompt=step_back_prompt)


In [ ]:
# example query over the understanding climate change dataset
original_query = "NHững ảnh hưởng của biến đổi khí hậu lên môi trường"
step_back_query = generate_step_back_query(original_query)


In [ ]:
print("Original query:", original_query)
print("\nStep-back query:", step_back_query['text'])

Original query: NHững ảnh hưởng của biến đổi khí hậu lên môi trường

Step-back query: Truy vấn bước lùi: "Các yếu tố và tác động của biến đổi khí hậu đối với hệ sinh thái và môi trường toàn cầu"


# **3- Sub-query Decomposition: Breaking complex queries into simpler sub-queries.**

In [ ]:
# Create a prompt template for sub-query decomposition
subquery_decomposition_template = """Bạn là trợ lý AI được giao nhiệm vụ phân tích các truy vấn phức tạp thành các truy vấn con đơn giản hơn cho hệ thống RAG.
Với truy vấn gốc, hãy phân tích truy vấn đó thành 2-4 truy vấn con đơn giản hơn, khi trả lời cùng nhau, sẽ cung cấp phản hồi toàn diện cho truy vấn gốc.

Truy vấn gốc: {original_query}

ví dụ: Biến đổi khí hậu tác động như thế nào đến môi trường?

Truy vấn con:
1. Biến đổi khí hậu tác động như thế nào đến đa dạng sinh học?
2. Biến đổi khí hậu ảnh hưởng đến đại dương như thế nào?
3. Biến đổi khí hậu tác động như thế nào đến nông nghiệp?
4. Biến đổi khí hậu tác động như thế nào đến sức khỏe con người?"""


subquery_decomposition_prompt = PromptTemplate(
    input_variables=["original_query"],
    template=subquery_decomposition_template
)

# Create an LLMChain for sub-query decomposition
subquery_decomposer_chain = subquery_decomposition_prompt | model

def decompose_query(original_query: str):
    """
    Decompose the original query into simpler sub-queries.

    Args:
    original_query (str): The original complex query

    Returns:
    List[str]: A list of simpler sub-queries
    """
    response = subquery_decomposer_chain.invoke(original_query).content
    sub_queries = [q.strip() for q in response.split('\n') if q.strip() and not q.strip().startswith('Sub-queries:')]
    return sub_queries

In [ ]:
# example query over the understanding climate change dataset
original_query = "Ảnh hưởng của Internet đến giới trẻ hiện nay?"
sub_queries = decompose_query(original_query)
print("\nSub-queries:")
for i, sub_query in enumerate(sub_queries, 1):
    print(sub_query)


Sub-queries:
Truy vấn gốc: Ảnh hưởng của Internet đến giới trẻ hiện nay?
Truy vấn con:
1. Internet ảnh hưởng đến cách giao tiếp và kết nối của giới trẻ như thế nào?
2. Internet tác động đến việc học tập và tiếp thu kiến thức của giới trẻ ra sao?
3. Internet ảnh hưởng đến sức khỏe tâm lý và cảm xúc của giới trẻ như thế nào?
4. Internet có tác động gì đến thói quen tiêu dùng và sở thích giải trí của giới trẻ?


# **Concat Chat History**

In [ ]:

summary_template = """Bạn là trợ lý AI được giao nhiệm vụ tóm tắt nội dung của câu truy vấn
Hãy đảm bảo rằng nội dung sau khi được tóm tắt sẽ ngắn gọn hơn mà vẫn giữ nguyên ngữ nghĩa
Nội dung gốc: {original_query}
Nội dung sau khi tóm tắt: """

history_template = PromptTemplate(
    input_variables=["original_query"],
    template=summary_template
)

history_llm = history_template | model



In [ ]:
query_1 = "Mẫu điện thoại iphone X của Apple được ra mắt vào ngày nào?"
query_2 = " 11 "
query_3 = " 13 thì sao"

concatted_query = query_1 + "," + query_2 +", " + query_3
concatted_query

'Mẫu điện thoại iphone X của Apple được ra mắt vào ngày nào?, 11 ,  13 thì sao'

In [ ]:

sub_queries = decompose_query(concatted_query)
print("\nSub-queries:")
for i, sub_query in enumerate(sub_queries, 1):
    print(sub_query)


Sub-queries:
Truy vấn gốc: Mẫu điện thoại iPhone X của Apple được ra mắt vào ngày nào? 11, 13 thì sao?
Truy vấn con:
1. Mẫu điện thoại iPhone X của Apple được ra mắt vào ngày nào?
2. Có thông tin gì về các mẫu iPhone 11 và iPhone 13 không?
3. So sánh các tính năng chính giữa iPhone X, iPhone 11 và iPhone 13 là gì?
4. Giá cả và thời điểm phát hành của iPhone 11 và iPhone 13 là gì?


In [ ]:
concatted_query = sub_queries[0] + " " +  sub_queries[1] + " " + sub_queries[2] + " " + sub_queries[3]

In [ ]:
response = history_llm.invoke(concatted_query)
response.content

'iPhone X của Apple ra mắt ngày nào? Có thông tin gì về iPhone 11 và 13 không?'